In [3]:
import random
import time
from IPython.display import clear_output
from itertools import cycle
import copy
from modulos.classes import *
from modulos.funcoes import *

In [4]:
comprimento_colunas = [2,4,6,8,10,12,10,8,6,4,2]
lista_jogadores = [Jogador('Gustavo'), Jogador('Pla')]

configuracao_jogo = Configuracao_jogo( 3, lista_jogadores , comprimento_colunas)

gerenciador = Gerenciador_partida(configuracao_jogo) 

gerenciador.iniciar_jogo()


---- Inicio da jogada ----
jogador_ativo
Gustavo
decisao.valores_escolhidos
(6, 6)
Status tabuleiro
[2]marcador: 0 comprimento: 2 - Gustavo 0 - Pla 0
[3]marcador: 0 comprimento: 4 - Gustavo 0 - Pla 0
[4]marcador: 0 comprimento: 6 - Gustavo 0 - Pla 0
[5]marcador: 0 comprimento: 8 - Gustavo 0 - Pla 0
[6]marcador: 1 comprimento: 10 - Gustavo 0 - Pla 0
[7]marcador: 0 comprimento: 12 - Gustavo 0 - Pla 0
[8]marcador: 0 comprimento: 10 - Gustavo 0 - Pla 0
[9]marcador: 0 comprimento: 8 - Gustavo 0 - Pla 0
[10]marcador: 0 comprimento: 6 - Gustavo 0 - Pla 0
[11]marcador: 0 comprimento: 4 - Gustavo 0 - Pla 0
[12]marcador: 0 comprimento: 2 - Gustavo 0 - Pla 0

---- Inicio da jogada ----
jogador_ativo
Gustavo
decisao.valores_escolhidos
(8, 7)
Status tabuleiro
[2]marcador: 0 comprimento: 2 - Gustavo 0 - Pla 0
[3]marcador: 0 comprimento: 4 - Gustavo 0 - Pla 0
[4]marcador: 0 comprimento: 6 - Gustavo 0 - Pla 0
[5]marcador: 0 comprimento: 8 - Gustavo 0 - Pla 0
[6]marcador: 1 comprimento: 10 - Gustavo 0 

In [5]:
def rodar_jogo(Configuracao_jogo):
    #preparando as classes
    tabuleiro=Tabuleiro(Configuracao_jogo)
    pontos=[]
    lista_jogadores = Configuracao_jogo.lista_jogadores
    jogador_ativo = lista_jogadores[0]
    
    #choose your move
    def executar_decisao(decisao_do_jogador , jogador_ativo):
        '''execute the chosen move'''
        #make your move
        executou_movimento=False
        #### #for valor in decisao_do_jogador:
        while(decisao_do_jogador!=[]):
            for i in range (2,13): ###acho que podemos percorrer o decisao_do_jogador ao invés de range(2,13).
                if decisao_do_jogador==[]:
                    break
                elif tabuleiro.pode_adicionar_marcador_na_coluna(decisao_do_jogador[0]): #se tem marcardor disponível, bota marcador
                    print(f'Colocou marcador na coluna: {decisao_do_jogador[0]}')
                    print(decisao_do_jogador[0], tabuleiro.colunas[decisao_do_jogador[0]].marcador)

                    tabuleiro.start_marcador_coluna(decisao_do_jogador[0],jogador_ativo)
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    executou_movimento=True

                elif decisao_do_jogador[0] in tabuleiro.lista_colunas_com_marcadores(): #se tem marcador posicionado, avança marcador
                    print(f'Avançou marcador na coluna: {decisao_do_jogador[0]}')
                    print(decisao_do_jogador[0], tabuleiro.colunas[decisao_do_jogador[0]].marcador)

                    tabuleiro.avancar_marcador_coluna(decisao_do_jogador[0])
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    executou_movimento=True
                
                else:
                    print('')
                    print('')
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    
        if executou_movimento==False:
            tabuleiro.limpar_tabuleiro()
            return False
        return True
                    
    def passar_turno(jogador_ativo):
        '''voluntarily end your turn'''
        tabuleiro.gravar_avancos(jogador_ativo)
            
    for i in range(500):
        
        time.sleep(0.1)
        clear_output(wait=True)
        dados=tabuleiro.rolar_dados()
        decisao_do_jogador=list(dados[random.randrange(0,len(dados))]) ### Nessa versão estamos escolhendo uma opção ao acaso ###
        print(f'jogador {jogador_ativo.nome}')
        print(f'jogador {jogador_ativo.nome} escolheu: {decisao_do_jogador}')
        if not executar_decisao(decisao_do_jogador , jogador_ativo): ### a jogada deu ruim, o jogador ativo encerra o turno.
            print(f'jogador {jogador_ativo.nome} Explodiu')
            if jogador_ativo==lista_jogadores[0]: 
                jogador_ativo=lista_jogadores[1]
            else:
                jogador_ativo=lista_jogadores[0]            
        elif len(tabuleiro.lista_colunas_com_marcadores())>=3:
            passar_turno(jogador_ativo)
            print(f'jogador {jogador_ativo.nome} Passou o turno')
            if jogador_ativo==lista_jogadores[0]:
                jogador_ativo=lista_jogadores[1]
            else:
                jogador_ativo=lista_jogadores[0]
        print(f'Tabuleiro: \ncolunas com marcadores: {tabuleiro.lista_colunas_com_marcadores()}')
        print(f'Colunas Bloqueadas: {tabuleiro.lista_colunas_bloqueadas()}')
        print(f'pontos:: {pontos}')
        for i in range (2,13):
            print(f'Coluna {i}: {list(map(lambda jogador: tabuleiro.colunas[i].posicao[jogador], tabuleiro.colunas[i].posicao))}')
        if len(tabuleiro.lista_colunas_bloqueadas())>=5:
            print(tabuleiro.lista_colunas_bloqueadas())
            break

In [6]:
n_colunas = [2,4,6,8,10,12,10,8,6,4,2]
lista_jogadores = [Jogador('Gustavo') , Jogador('Plá')]
configuracoes = Configuracao_jogo(3 , lista_jogadores , n_colunas)

rodar_jogo(configuracoes)

jogador Gustavo
jogador Gustavo escolheu: [9, 7]
Avançou marcador na coluna: 9
9 8
Colocou marcador na coluna: 7
7 0
jogador Gustavo Passou o turno
Tabuleiro: 
colunas com marcadores: []
Colunas Bloqueadas: [2, 4, 5, 6, 7, 9]
pontos:: []
Coluna 2: [3, 1]
Coluna 3: [4, 1]
Coluna 4: [1, 7]
Coluna 5: [2, 9]
Coluna 6: [11, 9]
Coluna 7: [13, 3]
Coluna 8: [10, 4]
Coluna 9: [9, 4]
Coluna 10: [4, 6]
Coluna 11: [0, 3]
Coluna 12: [2, 0]
[2, 4, 5, 6, 7, 9]
